In [3]:
import os

api_key = os.environ["OPENAI_API_KEY"]

In [1]:
import pandas as pd
import tiktoken
import os
import openai
import pickle
from langchain.embeddings import OpenAIEmbeddings

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader

In [3]:
with open('restaurent_docs.pickle', 'rb') as f:
    business_df = pickle.load(f)

In [16]:
with open('houston_rest_df.pickle', 'rb') as f:
    rest_df = pickle.load(f)

In [4]:
business_df.fillna(' ')

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,text,doc_information
0,Fadi's,"Fadi's, 6365 Westheimer Rd, Houston, TX 77057",0x8640c346cda31ecf:0xdccfb2cdf28f02dc,,29.737234,-95.496488,[Mediterranean restaurant],4.7,18,0,"[[Wednesday, 11AM–9PM], [Thursday, 11AM–9PM], ...","{'Service options': ['Takeout', 'Dine-in', 'De...",Closed ⋅ Opens 11AM Thu,,https://www.google.com/maps/place//data=!4m2!3...,Yes 🙌🏽. I got the curry chicken with a number ...,Mediterranean restaurant Takeout Dine-in Del...
1,Jus' Mac,"Jus' Mac, 5640 Westheimer Rd, Houston, TX 77056",0x8640c15f393c8e2b:0xcebbc17fcff328bf,,29.738526,-95.475659,[American restaurant],4.5,16,0,"[[Wednesday, 11:30AM–9PM], [Thursday, 11:30AM–...","{'Service options': ['Delivery', 'Takeout', 'D...",Closed ⋅ Opens 11:30AM Thu,,https://www.google.com/maps/place//data=!4m2!3...,This food isn’t worth two nickels rubbed toget...,American restaurant Delivery Takeout Dine-in...
2,chill works,"chill works, 3509 Elgin St Suite 200, Houston,...",0x8640bfa9613de5f1:0xf720305f9ba54d81,,29.728121,-95.349025,"[Frozen yogurt shop, Dessert shop, Frozen dess...",4.9,28,0,"[[Wednesday, 12–8PM], [Thursday, 12–8PM], [Fri...",{'From the business': ['Identifies as Black-ow...,Open ⋅ Closes 8PM,,https://www.google.com/maps/place//data=!4m2!3...,I brought four kids in who all wanted to taste...,Frozen yogurt shop Dessert shop Frozen desse...
3,Tacos Adrian's Puerto Vallarta,"Tacos Adrian's Puerto Vallarta, 9705 Synott Rd...",0x8640dd873932f3cb:0x1d55512b687664e,,29.674672,-95.613862,[Restaurant],4.4,28,0,"[[Wednesday, 2:30–11:30PM], [Thursday, 2:30–11...","{'Service options': ['Takeout', 'Dine-in', 'De...",Closed ⋅ Opens 2:30PM,,https://www.google.com/maps/place//data=!4m2!3...,GREEN SAUCE VERY SPICY. ONIONS VERY SPICY. GOO...,"Restaurant Takeout Dine-in Delivery, Lunch D..."
4,Fire Wings Westheimer,"Fire Wings Westheimer, 8401 Westheimer Rd #110...",0x8640c35c5577dd3b:0x6f468afe463fd1aa,,29.736600,-95.514498,"[Chicken wings restaurant, Restaurant]",4.0,75,2,"[[Wednesday, 11AM–10PM], [Thursday, 11AM–10PM]...","{'Service options': ['Delivery', 'Takeout', 'D...",Closed ⋅ Opens 11AM,,https://www.google.com/maps/place//data=!4m2!3...,I got 4 varieties of boneless wings. All good ...,Chicken wings restaurant Restaurant Delivery...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,Pappasito's Cantina,"Pappasito's Cantina, Hilton Americas, 1600 Lam...",0x8640bf22f4a3137d:0xc9c89014d4d23168,Tex-Mex dishes & margaritas are the stars at t...,29.751927,-95.361120,[Mexican restaurant],4.2,1894,2,"[[Thursday, 4–9PM], [Friday, 4–9PM], [Saturday...","{'Service options': ['Outdoor seating', 'Curbs...",,"[0x8640bf23db6b3db1:0xdd2ab824e8bc3d77, 0x8640...",https://www.google.com/maps/place//data=!4m2!3...,I am a big fan of places that have a great atm...,Tex-Mex dishes & margaritas are the stars at t...
6995,Krispy Kreme,"Krispy Kreme, 8611 Westheimer Rd, Houston, TX ...",0x8640c3a739a4a165:0x51d500b092866a00,Longtime chain serving a variety of donut flav...,29.736951,-95.518211,"[Donut shop, Coffee shop, Dessert shop, Fast f...",3.9,1767,1,"[[Thursday, 6AM–10PM], [Friday, 6AM–11PM], [Sa...","{'Service options': ['Curbside pickup', 'Deliv...",,"[0x8641216a8691951d:0xc21f63d4676e5750, 0x8640...",https://www.google.com/maps/place//data=!4m2!3...,Krispy Kreme is my favorite donut shop! I love...,Longtime chain serving a variety of donut flav...
6996,E-Tao Asian Eatery,"E-Tao Asian Eatery, 5135 W Alabama St #7295, H...",0x8640c16ef4e7ffef:0xb4aaa7594e77f42f,Contemporary eatery in a retail mall offering ...,29.736558,-95.463934,"[Asian restaurant, Restaurant]",4.0,166,0,"[[Thursday, 10AM–9PM], [Friday, 10AM–9PM], [Sa...",{'Service options': ['Delivery']},,"[0x8640c16a2a68446f:0x71dd67a5610c9b2a, 0x8640...",https://www.google.com/maps/place//data=!4m2!3...,"Poor service, bad taste in food, and items on ...",Contemporary

In [5]:
columns_to_drop = ['relative_results', 'text', 'url', 'doc_information']

business_df.drop(columns=columns_to_drop, inplace=True)

In [6]:
docs_df = business_df

In [7]:
def concat_dict_values(d):
    if d is None:
        return ''
    return ', '.join([f"{' '.join(value)}" for key, value in d.items()])


docs_df['combined_info'] = 'Name: ' +docs_df['name']+' Description: '+ docs_df['description'].fillna(' ') + ' Category: ' + docs_df['category'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x)) + ' Service Options: ' + docs_df['MISC'].apply(concat_dict_values) 

In [8]:
short_df = docs_df

In [10]:
short_df.fillna(' ', inplace=True)

In [9]:
short_df.isna().any()

name              False
address           False
gmap_id           False
description        True
latitude          False
longitude         False
category          False
avg_rating        False
num_of_reviews    False
price             False
hours              True
MISC               True
state              True
combined_info     False
dtype: bool

In [11]:
short_df = short_df.astype(str)


In [12]:
from langchain_community.document_loaders import DataFrameLoader
loader = DataFrameLoader(short_df, page_content_column="combined_info")
data = loader.load()

In [13]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [14]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
llm = OpenAI(openai_api_key=api_key)

#Vector DB
docsearch = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db_res')

/Users/nachiket/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/nachiket/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [23]:
query = "I'm looking for indian food. What could you suggest to me?"
docs = docsearch.similarity_search(query, k=3)
docs

[Document(page_content='Name: Shiva Indian Restaurant Description: Laid-back eatery serving a variety of familiar dishes such as tandoori chicken, saag paneer & naan. Category: Indian restaurant Restaurant Service Options: No-contact delivery Delivery Takeout Dine-in, Mask required, Great tea selection, Lunch Dinner Solo dining, Wheelchair accessible entrance, All you can eat Coffee Comfort food Halal food Healthy options Small plates Vegetarian options, Catering Dessert, Good for kids High chairs, Casual Cozy, College students Groups Tourists, Accepts reservations', metadata={'MISC': "{'Service options': ['No-contact delivery', 'Delivery', 'Takeout', 'Dine-in'], 'Health & safety': ['Mask required'], 'Highlights': ['Great tea selection'], 'Popular for': ['Lunch', 'Dinner', 'Solo dining'], 'Accessibility': ['Wheelchair accessible entrance'], 'Offerings': ['All you can eat', 'Coffee', 'Comfort food', 'Halal food', 'Healthy options', 'Small plates', 'Vegetarian options'], 'Dining options'

In [5]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
llm = OpenAI(openai_api_key=api_key)

vectordb = Chroma(persist_directory='./chroma_db_res', embedding_function=embeddings)

retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

/Users/nachiket/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [24]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [25]:
qa = RetrievalQA.from_chain_type(llm,
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=True)

In [6]:
query = "I'm looking for indian food. What could you suggest to me?"
result = qa({"query": query})
result['result']

/Users/nachiket/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


" \nBased on the given information, I would suggest Annam Indian Cuisine or Shiva Indian Restaurant for dine-in or takeout options. For a more casual and quick dining experience, Spice Indian Veg Cuisine Food Truck would be a good choice. If you are looking for a no-frills and budget-friendly option, India's Restaurant would be a good choice with their lunch buffet."

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

template = """You are a restaurent recommender system that help users to find restaurents that match their preferences. 
Use the following pieces of context to answer the question at the end. 
For each question, suggest three restaurents, with a short description and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""


PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0) 

qa = RetrievalQA.from_chain_type(llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

query = "I like mexican food, suggest me something similar but not mexican"
result = qa({'query':query})
print(result['result'])

1. Panda Express - Panda Express offers American Chinese cuisine with a variety of flavorful dishes like Orange Chicken and Beijing Beef. The user might enjoy the bold flavors and unique dishes that Chinese cuisine has to offer.

2. Tokyo Joe's - Tokyo Joe's serves up fresh and healthy Asian-inspired dishes like sushi rolls, rice bowls, and salads. The user might appreciate the fresh ingredients and lighter options that Japanese cuisine provides.

3. Indian Curry House - Indian Curry House offers a wide range of traditional Indian dishes like curry, biryani, and tandoori specialties. The user might enjoy the aromatic spices and rich flavors that Indian cuisine is known for.
